# Preparation

In [ ]:
import math
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sys.path.insert(0,'../../../../') # The simulator package and package needed for analysis 
                                  # (which are imported below) are in a parent folder

# Modified package from https://github.com/BilHim/trafficSimulator
from trafficSimulator import *
# Functions needed for simulation transition calculation
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

In [ ]:
vehicle_rate = 45

## Simulation setup

### Simulation params setup

In [ ]:
# Specify the labels of the road segments in each route
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

# Levels of speed limit for fast and slow roads
# expressed as a proportion of the v_max set above
fast_track_factor, slow_track_factor = 1, 0.33 

# Length of the stop zone
stop_distance = 25 

# Waiting time for fast and slow roads (here set as the same)
fast_wait_time = 4
slow_wait_time = 4

# Total number of vehicles
vehicle_limit = 80

# Store vehicle specs to be input in the simulation package
vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

# Empty dataframe to store the simulation data
df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

# Initialize the probabilities for choosing each route for the 1st round;
# each route has an equal chance of being selected
vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

### Topology setup

In [ ]:
# Coordinates of the four corners of the traffic network; all four corners
# lie on either the x or the y axis
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

# Building the road lines with corresponding parameters
left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

## For the connections of the topology, we can represent with concrete lines, but
# the vehicles will not actually go through these routes; 
# they are  for demonstration purposes
connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

### Utility params setup

In [ ]:
# Exploration factor in UCB alogorithm
delta = 0.25

# Penalty factor
alpha = 5

## Function needed for running simulation

In [ ]:
 def run_simulation(round_number, 
                    all_routes, 
                    vehicle_limit, 
                    df, 
                    vehicle_preferences,
                    vehicle_rate, 
                    vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [ ]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 3min 26s, sys: 5.14 s, total: 3min 31s
Wall time: 3min 30s


In [ ]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[0, 1]",76.3,"[-999, 13]","[0, 0.05]","[2.9999999999999973, 0]"
1,1,"[0, 1]",82.7,"[0, 16]","[4.199999999999993, 0]","[2.25, 0]"
2,2,"[0, 1]",85.7,"[1, 1]","[4.499999999999992, 0]","[0, 0]"
3,3,"[0, 3]",119.0,"[2, 17]","[7.099999999999983, 0]","[0, 0]"
4,4,"[0, 3]",123.95,"[3, 3]","[9.700000000000003, 0]","[0, 0]"
...,...,...,...,...,...,...
75,75,"[0, 1]",206.85,"[73, 69]","[66.74999999999835, 0]","[0, 0]"
76,76,"[0, 1]",210.6,"[75, 75]","[69.54999999999819, 0]","[0, 0]"
77,77,"[2, 3]",245.45,"[74, 74]","[74.7999999999979, 0]","[0, 0]"
78,78,"[0, 1]",213.0,"[76, 76]","[71.59999999999808, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [ ]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[0, 1]",76.3,"[-999, 13]","[0, 0.05]","[2.9999999999999973, 0]",1
1,1,"[0, 1]",82.7,"[0, 16]","[4.199999999999993, 0]","[2.25, 0]",1
2,2,"[0, 1]",85.7,"[1, 1]","[4.499999999999992, 0]","[0, 0]",1
3,3,"[0, 3]",119.0,"[2, 17]","[7.099999999999983, 0]","[0, 0]",1
4,4,"[0, 3]",123.95,"[3, 3]","[9.700000000000003, 0]","[0, 0]",1
5,5,"[0, 1]",96.95,"[4, 22]","[12.30000000000004, 0]","[0, 0]",1
6,6,"[0, 1]",106.9,"[5, 23]","[17.000000000000107, 0.05]","[2.0500000000000007, 0]",1
7,7,"[0, 1]",108.9,"[6, 6]","[17.600000000000115, 0]","[0, 0]",1
8,8,"[0, 3]",145.35,"[7, 26]","[20.200000000000152, 0]","[0, 0]",1
9,9,"[2, 1]",36.45,"[-999, -999]","[0, 0]","[0, 0]",1


### Set up dataframe

In [ ]:
utility_df = pd.read_csv("../2.3.1.1 Zero alpha/Saved_data/Utility_data/Round_1.csv")

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],4.476024,1.020013
1,1,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.404149,0.997322
2,2,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],4.921898,1.226696
3,3,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.535169,1.113273
4,4,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.404324,0.987959


## Summary of the 1st round

In [ ]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,1,4.25,1,76.30
1,1,1,4.50,1,82.70
2,2,1,7.10,1,85.70
3,3,0,9.70,1,119.00
4,4,0,12.30,1,123.95
5,5,1,17.00,1,96.95
6,6,1,17.60,1,106.90
7,7,1,20.20,1,108.90
8,8,0,16.10,1,145.35
9,9,3,1.85,1,36.45


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[16.651092223153956, -7.67423088107452, 16.651...","[0.3333333333303036, 9.089189342986218e-12, 0....",[1],4.476024,1.020013
1,1,"[16.651092223153956, -7.488254285799599, 16.65...","[0.3333333333296843, 1.0946965759737035e-11, 0...",[1],5.404149,0.997322
2,2,"[16.651092223153956, -16.78487616812741, 16.65...","[0.333333333333333, 1.004204529296393e-15, 0.3...",[1],4.921898,1.226696
3,3,"[-12.795903674869743, 16.651092223153956, 16.6...","[5.422641808394901e-14, 0.3333333333333153, 0....",[0],5.535169,1.113273
4,4,"[-8.612123124379012, 16.651092223153956, 16.65...","[3.557981820375241e-12, 0.33333333333214726, 0...",[0],5.404324,0.987959
5,5,"[16.651092223153956, -3.987499034917384, 16.65...","[0.3333333332124041, 3.6278761080390054e-10, 0...",[1],5.402144,0.733822
6,6,"[16.651092223153956, -4.8042963723972765, 16.6...","[0.3333333332799014, 1.6029569951861157e-10, 0...",[1],4.608463,0.797110
7,7,"[16.651092223153956, -10.685994782860165, 16.6...","[0.3333333333331842, 4.4723183210758465e-13, 0...",[1],4.665863,1.045836
8,8,"[-4.849089649931055, 16.651092223153956, 16.65...","[1.5327396698721427e-10, 0.333333333282242, 0....",[0],4.782966,0.796192
9,9,"[16.651092223153956, 16.651092223153956, 16.65...","[0.3333333333125659, 0.3333333333125659, 0.333...",[3],4.666469,1.003395


In [ ]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [ ]:
%%time

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 50

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, 
                                all_routes, 
                                vehicle_limit, 
                                df, 
                                vehicle_preferences,
                                vehicle_rate, 
                                vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    
    vehicle_preferences = dict(zip(range(vehicle_limit), 
                                   utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)

CPU times: user 2h 19min 8s, sys: 3min 34s, total: 2h 22min 42s
Wall time: 2h 17min 45s
